In [1]:
import os
import sys
import shutil
import numpy as np
import pandas as pd
import pydicom as dicom
import matplotlib.pylab as plt
import SimpleITK as sitk
from datetime import datetime


In [2]:
%env SITK_SHOW_COMMAND "/home/andres/Downloads/Slicer-4.10.2-linux-amd64/Slicer"
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
dicom.config.convert_wrong_length_to_UN = True

env: SITK_SHOW_COMMAND="/home/andres/Downloads/Slicer-4.10.2-linux-amd64/Slicer"


In [281]:
parent_dir = os.path.abspath('..')
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from Utils import utils

ImportError: cannot import name 'display_full' from 'Utils' (/home/workstation/Ashish/UCAN-PET-CT-image-data-handling-pipeline/Utils/__init__.py)

In [4]:
config = utils.read_config()
config

{'common': {'paths': {'source_path_wd': 'E:/U-CAN-Lymfom_A',
   'source_path_bd': 'F:/ucan_lymfom'}},
 'selection': {'filenames': {'rejected_folder_filename': 'Rejected_exams_from_U-CAN-Lymfom.xlsx',
   'sourcefiltered_folder_filename': 'SourceFiltered_exams_from_U-CAN-Lymfom.xlsx',
   'incomplete_folders_filename_initial': 'No_PTorCT_exams_from_U-CAN-Lymfom1.xlsx',
   'incomplete_folders_filename_final': 'No_PTorCT_exams_from_U-CAN-Lymfom2.xlsx',
   'selected_folders_beforefiltering_filename': 'Selected_exams_beforefiltering_from_U-CAN-Lymfom.xlsx',
   'selected_folders_afterfiltering_filename': 'Selected_exams_afterfiltering_from_U-CAN-Lymfom.xlsx',
   'final_selected_images_filename': 'FinalSelected_exams_from_U-CAN-Lymfom.xlsx',
   'list_of_distorted_images_filename': 'Distorted_exams_from_U-CAN-Lymfom.xlsx'}},
 'metadata': {'filenames': {'final_selected_images_filename': 'Finalized_dataset.xlsx'}},
 'resampling': {'voxel_spacing': [2.0364201068878174, 2.0364201068878174, 3.0],
  '

In [5]:
#source_path_wd = "/media/andres/T7 Shield1/U-CAN-Lymfom_A/Selected_for_UCAN_project"
#source_path_bd = "/media/andres/T7 Shield/ucan_lymfom"

source_path_wd = config['common']['paths']['source_path_wd']
source_path_bd = config['common']['paths']['source_path_bd']

master_data_wd_filename = config['metadata']['filenames']['final_selected_images_filename']
master_data_bd_filename = config['metadata']['filenames']['final_selected_images_filename']


In [6]:
final_selected_images_filename = config['selection']['filenames']['final_selected_images_filename']
final_selected_folders_bd = source_path_bd + '/' + final_selected_images_filename

In [37]:
# master_data for black disk
print("Reading master data for black disk")
#master_data_bd = pd.read_excel(os.path.join(source_path_bd, master_data_wd_filename))
master_data_bd = pd.read_excel(source_path_bd + "/Excel_files/" + master_data_wd_filename, parse_dates=["scan_date"])
#master_data_wd[['system', 'npr', 'scan_date']]= master_data_wd['patient_directory'].str.split(pat='_|-', n=2, expand=True)
print(master_data_bd.shape)

master_data_bd.head(2)

Reading master data for black disk
(2264, 45)


,Unnamed: 0,Unnamed: 0_x,directory,source_directory,patient_directory,PET-CT_info,system,npr,scan_date,Unnamed: 0_y,dicom_img,patient_id,patient_age,patient_weight,patient_size,imgsz_x,imgsz_y,num_slices,voxsz_x,voxsz_y,slice_thickness,image_size,voxel_size,img_pos,img_orient,for_uid,att_corr,recons_method,image_type,aquisition_dt,aquisition_time,study_desc,series_desc,protocol,corr_img,modality,manufacturer,manufacturer_model,radiopharmaceutical,radiopharmaceutical_volume,radiopharmaceutical_start_time,radionuclide_total_dose,radionuclide_half_life,radionuclide_positron_fraction,radiopharmaceutical_start_date_time
0,0,2096,F:/ucan_lymfom/ASPTCTX0001_lpr385705046400-20140313/CT-20140313-152551-6.1_UAS-WB-FDG-3D-STANDARD_CT_RECON-3.750000mm,F:/ucan_lymfom,ASPTCTX0001_lpr385705046400-20140313,CT-20140313-152551-6.1_UAS-WB-FDG-3D-STANDARD_CT_RECON-3.750000mm,ASPTCTX0001,lpr385705046400,2014-03-13,13743,1.2.840.113619.2.55.3.4240671514.332.1394175950.81.1,lpr385705046400,33.0,'69.0','1.73',512,512,215,0.976562,0.976562,3.75,"(512, 512, 215)","(0.976562, 0.976562, 3.75)","[-250.000, -250.000, 167.080]","[1.000000, 0.000000, 0.000000, 0.000000, 1.000000, 0.000000]",1.2.840.113619.2.55.3.4240671514.332.1394175949.830.13830.1,NaN,NaN,"['ORIGINAL', 'PRIMARY', 'AXIAL']",'20140313','152608','FDG WB 1 h','STANDARD CT RECON','6.1 UAS-WB-FDG-3D',NaN,CT,'GE MEDICAL SYSTEMS','Discovery STE',NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2095,F:/ucan_lymfom/ASPTCTX0001_lpr385705046400-20140313/PT-20140313-152728-UAS-WB-FDG-3D-UAS-WB-FDG-3D-AC-3.2700mm,F:/ucan_lymfom,ASPTCTX0001_lpr385705046400-20140313,PT-20140313-152728-UAS-WB-FDG-3D-UAS-WB-FDG-3D-AC-3.2700mm,ASPTCTX0001,lpr385705046400,2014-03-13,13745,1.2.840.113619.2.131.4240671514.1394722651.463822,lpr385705046400,33.0,'69.0','1.73',128,128,247,3.906250,3.906250,3.27,"(128, 128, 247)","(3.90625, 3.90625, 3.27)","[-248.04687500000, -248.04687500000, 169.020004272460]","[1, 0, 0, 0, 1, -0]",1.2.840.113619.2.55.3.4240671514.332.1394175949.830.13830.1,"'measured,, 0.096000 cm-1,'",'3D IR',"['ORIGINAL', 'PRIMARY']",'20140313','154238','FDG WB 1 h','UAS-WB-FDG-3D-AC','UAS-WB-FDG-3D',"['DECY', 'ATTN', 'SCAT', 'DTIM', 'RAN', 'DCAL', 'SLSENS', 'NORM']",PT,'GE MEDICAL SYSTEMS','Discovery STE','FDG -- fluorodeoxyglucose','0.0','141400.00','259344912.0','6588.0','0.97000002861023','20140313141400.00'


In [38]:
#(2264 / 25) * 2 -> 181.12 Gb

In [39]:
#misclassified_df = pd.read_excel(source_path_bd + '/MisclassifiedSelected_files.xlsx')
misclassified_df = pd.read_excel(source_path_bd + '//Excel_files/MisclassifiedSelected_files.xlsx')
misclassified_df[['source_directory', 'patient_directory', 'PET-CT_info']] = misclassified_df['directory'].str.rsplit(pat='/', n=2, expand=True)
misclassified_patient_directory_list = misclassified_df.patient_directory.to_list()
misclassified_df.head()

,Unnamed: 0,directory,count,source_directory,patient_directory,PET-CT_info
0,284,F:/ucan_lymfom/ASPTCTX0001_npr196644347217-20161125/CT-20161125-114603-6.3_UAS-WB-FDG-3D_CT_VENFAS-WB_VEN_AX-0.958984mm,1,F:/ucan_lymfom,ASPTCTX0001_npr196644347217-20161125,CT-20161125-114603-6.3_UAS-WB-FDG-3D_CT_VENFAS-WB_VEN_AX-0.958984mm
1,750,F:/ucan_lymfom/ASPTCTX0001_npr338511978351-20170201/CT-20170201-122546-6.3_UAS-WB-FDG-3D_CT_VENFAS-WB_VEN_AX-0.953125mm,1,F:/ucan_lymfom,ASPTCTX0001_npr338511978351-20170201,CT-20170201-122546-6.3_UAS-WB-FDG-3D_CT_VENFAS-WB_VEN_AX-0.953125mm
2,1398,F:/ucan_lymfom/ASPTCTX0001_npr567278394863-20151103/CT-20151103-130218-6.3_UAS-WB-FDG-3D_CT_THORAX-BUK_VENFAS_20141209-HELKROPP_VENFAS_AX_1MM-0.976562mm,1,F:/ucan_lymfom,ASPTCTX0001_npr567278394863-20151103,CT-20151103-130218-6.3_UAS-WB-FDG-3D_CT_THORAX-BUK_VENFAS_20141209-HELKROPP_VENFAS_AX_1MM-0.976562mm
3,1412,F:/ucan_lymfom/ASPTCTX0001_npr574436824692-20170222/CT-20170222-104855-6.6_UAS-WB-FDG-3D_VENFASTHORAX_INSP_20160120-WB_VEN_AX-0.951172mm,1,F:/ucan_lymfom,ASPTCTX0001_npr574436824692-20170222,CT-20170222-104855-6.6_UAS-WB-FDG-3D_VENFASTHORAX_INSP_20160120-WB_VEN_AX-0.951172mm
4,1682,F:/ucan_lymfom/ASPTCTX0001_npr676639996732-20180928/CT-20180928-150052-6.3_UAS-WB-FDG-3D_CT_VENFAS-WB_VEN_AX-0.976562mm,1,F:/ucan_lymfom,ASPTCTX0001_npr676639996732-20180928,CT-20180928-150052-6.3_UAS-WB-FDG-3D_CT_VENFAS-WB_VEN_AX-0.976562mm


In [40]:
master_data_bd[np.logical_and(master_data_bd.num_slices >200,master_data_bd.num_slices < 250)].head(2) # 488

,Unnamed: 0,Unnamed: 0_x,directory,source_directory,patient_directory,PET-CT_info,system,npr,scan_date,Unnamed: 0_y,dicom_img,patient_id,patient_age,patient_weight,patient_size,imgsz_x,imgsz_y,num_slices,voxsz_x,voxsz_y,slice_thickness,image_size,voxel_size,img_pos,img_orient,for_uid,att_corr,recons_method,image_type,aquisition_dt,aquisition_time,study_desc,series_desc,protocol,corr_img,modality,manufacturer,manufacturer_model,radiopharmaceutical,radiopharmaceutical_volume,radiopharmaceutical_start_time,radionuclide_total_dose,radionuclide_half_life,radionuclide_positron_fraction,radiopharmaceutical_start_date_time
0,0,2096,F:/ucan_lymfom/ASPTCTX0001_lpr385705046400-20140313/CT-20140313-152551-6.1_UAS-WB-FDG-3D-STANDARD_CT_RECON-3.750000mm,F:/ucan_lymfom,ASPTCTX0001_lpr385705046400-20140313,CT-20140313-152551-6.1_UAS-WB-FDG-3D-STANDARD_CT_RECON-3.750000mm,ASPTCTX0001,lpr385705046400,2014-03-13,13743,1.2.840.113619.2.55.3.4240671514.332.1394175950.81.1,lpr385705046400,33.0,'69.0','1.73',512,512,215,0.976562,0.976562,3.75,"(512, 512, 215)","(0.976562, 0.976562, 3.75)","[-250.000, -250.000, 167.080]","[1.000000, 0.000000, 0.000000, 0.000000, 1.000000, 0.000000]",1.2.840.113619.2.55.3.4240671514.332.1394175949.830.13830.1,NaN,NaN,"['ORIGINAL', 'PRIMARY', 'AXIAL']",'20140313','152608','FDG WB 1 h','STANDARD CT RECON','6.1 UAS-WB-FDG-3D',NaN,CT,'GE MEDICAL SYSTEMS','Discovery STE',NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2095,F:/ucan_lymfom/ASPTCTX0001_lpr385705046400-20140313/PT-20140313-152728-UAS-WB-FDG-3D-UAS-WB-FDG-3D-AC-3.2700mm,F:/ucan_lymfom,ASPTCTX0001_lpr385705046400-20140313,PT-20140313-152728-UAS-WB-FDG-3D-UAS-WB-FDG-3D-AC-3.2700mm,ASPTCTX0001,lpr385705046400,2014-03-13,13745,1.2.840.113619.2.131.4240671514.1394722651.463822,lpr385705046400,33.0,'69.0','1.73',128,128,247,3.906250,3.906250,3.27,"(128, 128, 247)","(3.90625, 3.90625, 3.27)","[-248.04687500000, -248.04687500000, 169.020004272460]","[1, 0, 0, 0, 1, -0]",1.2.840.113619.2.55.3.4240671514.332.1394175949.830.13830.1,"'measured,, 0.096000 cm-1,'",'3D IR',"['ORIGINAL', 'PRIMARY']",'20140313','154238','FDG WB 1 h','UAS-WB-FDG-3D-AC','UAS-WB-FDG-3D',"['DECY', 'ATTN', 'SCAT', 'DTIM', 'RAN', 'DCAL', 'SLSENS', 'NORM']",PT,'GE MEDICAL SYSTEMS','Discovery STE','FDG -- fluorodeoxyglucose','0.0','141400.00','259344912.0','6588.0','0.97000002861023','20140313141400.00'


In [41]:
exams_with_lt250 = master_data_bd[master_data_bd.num_slices < 250].patient_directory.to_list()
len(exams_with_lt250)

499

In [42]:
master_data_bd_filtered = master_data_bd[~np.logical_or(master_data_bd.patient_directory.isin(exams_with_lt250), master_data_bd.patient_directory.isin(misclassified_patient_directory_list))].copy().reset_index()
master_data_bd_filtered.shape

(1560, 46)

In [43]:
master_data_bd_filtered.patient_directory.nunique()

780

In [44]:
master_data_bd_filtered.patient_directory.nunique()

780

In [45]:
config_size = config['resampling']['image_size']
config_spacing = config['resampling']['voxel_spacing']
print('config_size: ', config_size)
print('config_spacing: ', config_spacing)

config_size:  [256, 256, 256]
config_spacing:  [2, 2, 3]


In [46]:
sample_SUV_image = sitk.ReadImage('F:/SUV.nii.gz')
print(sample_SUV_image.GetSize())
config_spacing = list(sample_SUV_image.GetSpacing())
config_spacing

(400, 400, 326)


[2.0364201068878174, 2.0364201068878174, 3.0]

In [47]:
#clip positive values at +3000
#set outside image 0 values to -1024

In [48]:
master_data_bd_filtered.head(2)

,index,Unnamed: 0,Unnamed: 0_x,directory,source_directory,patient_directory,PET-CT_info,system,npr,scan_date,Unnamed: 0_y,dicom_img,patient_id,patient_age,patient_weight,patient_size,imgsz_x,imgsz_y,num_slices,voxsz_x,voxsz_y,slice_thickness,image_size,voxel_size,img_pos,img_orient,for_uid,att_corr,recons_method,image_type,aquisition_dt,aquisition_time,study_desc,series_desc,protocol,corr_img,modality,manufacturer,manufacturer_model,radiopharmaceutical,radiopharmaceutical_volume,radiopharmaceutical_start_time,radionuclide_total_dose,radionuclide_half_life,radionuclide_positron_fraction,radiopharmaceutical_start_date_time
0,2,2,948,F:/ucan_lymfom/ASPTCTX0001_lpr415675513429-20190201/CT-20190201-103111-3.5_WB_FDG_CT_VENFAS-WB_Venfas_3_ax-3.000000mm,F:/ucan_lymfom,ASPTCTX0001_lpr415675513429-20190201,CT-20190201-103111-3.5_WB_FDG_CT_VENFAS-WB_Venfas_3_ax-3.000000mm,ASPTCTX0001,lpr415675513429,2019-02-01,14250,1.2.840.113619.2.5.91447938.12625.1549010475.262,lpr415675513429,32.0,'90.0','1.8',512,512,382,0.962891,0.962891,3.00,"(512, 512, 382)","(0.962891, 0.962891, 3.0)","[-249.237, -234.463, -127.315]","[1.000000, 0.000000, 0.000000, 0.000000, 1.000000, 0.000000]",1.2.840.113619.2.405.3.3842663696.892.1549005827.841.4795.1,NaN,NaN,"['DERIVED', 'SECONDARY', 'REFORMATTED', 'AVERAGE']",'20190201','102917.254016','PET FDG WB 1 h','WB Venfas 3 ax','3.5 WB_FDG CT VENFAS',NaN,CT,'GE MEDICAL SYSTEMS','Discovery MI',NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,3,947,F:/ucan_lymfom/ASPTCTX0001_lpr415675513429-20190201/PT-20190201-101500-3.5_WB_FDG_CT_VENFAS-QCFX-S_400-2.7900mm,F:/ucan_lymfom,ASPTCTX0001_lpr415675513429-20190201,PT-20190201-101500-3.5_WB_FDG_CT_VENFAS-QCFX-S_400-2.7900mm,ASPTCTX0001,lpr415675513429,2019-02-01,14246,1.2.840.113619.2.405.3842663696.1549013394.986841,lpr415675513429,32.0,'90.0','1.8',256,256,341,1.953125,1.953125,2.79,"(256, 256, 341)","(1.953125, 1.953125, 2.79)","[-249.02343750000, -249.02343750000, -128.24998474121]","[1, 0, 0, 0, 1, -0]",1.2.840.113619.2.405.3.3842663696.892.1549005827.841.4795.1,"'measured,, 0.000000 cm-1, attenuation smooth'",'QCFX',"['ORIGINAL', 'PRIMARY']",'20190201','102509','PET FDG WB 1 h','QCFX-S 400','3.5 WB_FDG CT VENFAS',"['DECY', 'ATTN', 'SCAT', 'DTIM', 'RANSNG', 'DCAL', 'SLSENS', 'NORM']",PT,'GE MEDICAL SYSTEMS','Discovery MI','FDG -- fluorodeoxyglucose','0.0','090500.00','264312992.0','6586.2001953125','0.96700000762939','20190201090500.00'


In [49]:
master_data_bd_filtered = master_data_bd_filtered.sort_values(by=["patient_directory"	,"PET-CT_info"], ascending=[True, False])
master_data_bd_filtered.head(2)

,index,Unnamed: 0,Unnamed: 0_x,directory,source_directory,patient_directory,PET-CT_info,system,npr,scan_date,Unnamed: 0_y,dicom_img,patient_id,patient_age,patient_weight,patient_size,imgsz_x,imgsz_y,num_slices,voxsz_x,voxsz_y,slice_thickness,image_size,voxel_size,img_pos,img_orient,for_uid,att_corr,recons_method,image_type,aquisition_dt,aquisition_time,study_desc,series_desc,protocol,corr_img,modality,manufacturer,manufacturer_model,radiopharmaceutical,radiopharmaceutical_volume,radiopharmaceutical_start_time,radionuclide_total_dose,radionuclide_half_life,radionuclide_positron_fraction,radiopharmaceutical_start_date_time
1,3,3,947,F:/ucan_lymfom/ASPTCTX0001_lpr415675513429-20190201/PT-20190201-101500-3.5_WB_FDG_CT_VENFAS-QCFX-S_400-2.7900mm,F:/ucan_lymfom,ASPTCTX0001_lpr415675513429-20190201,PT-20190201-101500-3.5_WB_FDG_CT_VENFAS-QCFX-S_400-2.7900mm,ASPTCTX0001,lpr415675513429,2019-02-01,14246,1.2.840.113619.2.405.3842663696.1549013394.986841,lpr415675513429,32.0,'90.0','1.8',256,256,341,1.953125,1.953125,2.79,"(256, 256, 341)","(1.953125, 1.953125, 2.79)","[-249.02343750000, -249.02343750000, -128.24998474121]","[1, 0, 0, 0, 1, -0]",1.2.840.113619.2.405.3.3842663696.892.1549005827.841.4795.1,"'measured,, 0.000000 cm-1, attenuation smooth'",'QCFX',"['ORIGINAL', 'PRIMARY']",'20190201','102509','PET FDG WB 1 h','QCFX-S 400','3.5 WB_FDG CT VENFAS',"['DECY', 'ATTN', 'SCAT', 'DTIM', 'RANSNG', 'DCAL', 'SLSENS', 'NORM']",PT,'GE MEDICAL SYSTEMS','Discovery MI','FDG -- fluorodeoxyglucose','0.0','090500.00','264312992.0','6586.2001953125','0.96700000762939','20190201090500.00'
0,2,2,948,F:/ucan_lymfom/ASPTCTX0001_lpr415675513429-20190201/CT-20190201-103111-3.5_WB_FDG_CT_VENFAS-WB_Venfas_3_ax-3.000000mm,F:/ucan_lymfom,ASPTCTX0001_lpr415675513429-20190201,CT-20190201-103111-3.5_WB_FDG_CT_VENFAS-WB_Venfas_3_ax-3.000000mm,ASPTCTX0001,lpr415675513429,2019-02-01,14250,1.2.840.113619.2.5.91447938.12625.1549010475.262,lpr415675513429,32.0,'90.0','1.8',512,512,382,0.962891,0.962891,3.00,"(512, 512, 382)","(0.962891, 0.962891, 3.0)","[-249.237, -234.463, -127.315]","[1.000000, 0.000000, 0.000000, 0.000000, 1.000000, 0.000000]",1.2.840.113619.2.405.3.3842663696.892.1549005827.841.4795.1,NaN,NaN,"['DERIVED', 'SECONDARY', 'REFORMATTED', 'AVERAGE']",'20190201','102917.254016','PET FDG WB 1 h','WB Venfas 3 ax','3.5 WB_FDG CT VENFAS',NaN,CT,'GE MEDICAL SYSTEMS','Discovery MI',NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Selected exams and clinical data matching

In [23]:
selectedexams_path = "/home/workstation/Ashish/UCAN/Final_Selected_exams_from_U-CAN-Lymfom.xlsx"
clinicaldata_path = "/home/workstation/Ashish/UCAN/clinical_data.xlsx"
#clinicaldata_path = "/home/workstation/Ashish/UCAN/U-CAN Lymfom - Ahlström 211021 - utlämnat_pnr_removed.xlsx"

#clinicaldata_path = "F:/ucan_lymfom/ClinicalData/U-CAN Lymfom - Ahlström 211021 - utlämnat_pnr_removed.xlsx"

In [24]:
master_data = pd.read_excel(selectedexams_path)
print(master_data.shape)
master_data.head()

(3656, 8)


,Unnamed: 0,directory,source_directory,patient_directory,PET-CT_info,system,npr,scan_date
0,0,/media/andres/T7 Shield/ucan_lymfom/ASPTCTX0001_npr999612862346-20210628/PT-20210628-112746-3.11_WB_F18_FDG_VENFAS__THORAX_INANDAD-QCFX-S_400_M.Free-2.8000mm,/media/andres/T7 Shield/ucan_lymfom,ASPTCTX0001_npr999612862346-20210628,PT-20210628-112746-3.11_WB_F18_FDG_VENFAS__THORAX_INANDAD-QCFX-S_400_M.Free-2.8000mm,ASPTCTX0001,npr999612862346,20210628
1,1,/media/andres/T7 Shield/ucan_lymfom/ASPTCTX0001_npr999612862346-20210628/CT-20210628-114331-3.11_WB_F18_FDG_VENFAS__THORAX_INANDAD-WB_Venfas_3_ax-3.000000mm,/media/andres/T7 Shield/ucan_lymfom,ASPTCTX0001_npr999612862346-20210628,CT-20210628-114331-3.11_WB_F18_FDG_VENFAS__THORAX_INANDAD-WB_Venfas_3_ax-3.000000mm,ASPTCTX0001,npr999612862346,20210628
2,2,/media/andres/T7 Shield/ucan_lymfom/ASPTCTX0001_npr999612862346-20210407/CT-20210407-105418-3.12_WB_F18_FDG_VENFAS__THORAX_INANDAD-WB_Venfas_0.6_ax-0.625000mm,/media/andres/T7 Shield/ucan_lymfom,ASPTCTX0001_npr999612862346-20210407,CT-20210407-105418-3.12_WB_F18_FDG_VENFAS__THORAX_INANDAD-WB_Venfas_0.6_ax-0.625000mm,ASPTCTX0001,npr999612862346,20210407
3,3,/media/andres/T7 Shield/ucan_lymfom/ASPTCTX0001_npr999612862346-20210407/PT-20210407-104228-3.12_WB_F18_FDG_VENFAS__THORAX_INANDAD-QCFX-S_400_M.Free-2.8000mm,/media/andres/T7 Shield/ucan_lymfom,ASPTCTX0001_npr999612862346-20210407,PT-20210407-104228-3.12_WB_F18_FDG_VENFAS__THORAX_INANDAD-QCFX-S_400_M.Free-2.8000mm,ASPTCTX0001,npr999612862346,20210407
4,4,/media/andres/T7 Shield/ucan_lymfom/ASPTCTX0001_npr999612862346-20210212/CT-20210212-130055-3.12_WB_FDG_VENFAS__THORAX_INANDAD-WB_Venfas_3_ax-3.000000mm,/media/andres/T7 Shield/ucan_lymfom,ASPTCTX0001_npr999612862346-20210212,CT-20210212-130055-3.12_WB_FDG_VENFAS__THORAX_INANDAD-WB_Venfas_3_ax-3.000000mm,ASPTCTX0001,npr999612862346,20210212


In [160]:
exams_data = master_data[["patient_directory", "system", "npr", "scan_date"]].copy().sort_values(by=["scan_date"]).reset_index(drop=True)
exams_data["scan_date"] = pd.to_datetime(exams_data["scan_date"], format="%Y%m%d")
exams_data = exams_data.dropna().reset_index(drop=True)
exams_data = exams_data.drop_duplicates().reset_index(drop=True)
#exams_data.scan_date = exams_data.scan_date.astype('datetime64[ns]')
print(exams_data.shape)
exams_data.head(2)

(1828, 4)


,patient_directory,system,npr,scan_date
0,ASPTCTX0001_npr601608371367-20100224,ASPTCTX0001,npr601608371367,2010-02-24
1,ASPTCTX0001_npr642326741392-20100301,ASPTCTX0001,npr642326741392,2010-03-01


In [161]:
exams_data.dtypes

patient_directory            object
system                       object
npr                          object
scan_date            datetime64[ns]
dtype: object

In [162]:
clinical_data = pd.read_excel(clinicaldata_path).sort_values(by=["consentDate"]).reset_index(drop=True)
clinical_data.columns = clinical_data.columns.str.strip()
print(clinical_data.shape)
clinical_data.head(2)

(1480, 39)


,personReference,inclusionYear,includingHospital,county,consentDate,consentComments,deceased,deceasedDate,sex,diagnosisGroups,DiaDate,stadium,Diagnosis,diagnosisComments,DiaDate 2,stadium 2,Diagnosis 2,diagnosisComments 2,TREATMENT,indication,startDate,endDate,typeDetails,OP/BIOPSYdate,type,tissueType,tissueTypeOther,OP/BIOPSYdate2,type 2,tissueType 2,surgeryComments 2,RELAPSE1,date,verifictationType,relapseComments,RELAPSE2,date 2,verifictationType 2,relapseComments 2
0,npr446854059583,2010.0,NaN,UPPSALA,2010-04-11,Två korrekta samtycken i KOVIS. 2010-04-11 + 2010-11-01,True,2015-01-03,MALE,HEMA,2010-10-28,NaN,C91.1 Kronisk lymfatisk leukemi av Bcellstyp [B-KLL],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,npr165959402726,2010.0,NaN,UPPSALA,2010-06-16,NaN,False,NaN,MALE,HEMA,2010-01-19,NaN,C91.1 Kronisk lymfatisk leukemi av Bcellstyp [B-KLL],KLL beh-lista,NaN,NaN,NaN,NaN,CHEMOTHERAPY,CURATIVE,2011-10-05,2012-05-22,Leukeran,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [163]:
clinical_data.consentDate.unique()

array(['2010-04-11', '2010-06-16', '2010-07-02', ..., '2021-10-06',
       '2021-10-11', nan], dtype=object)

In [164]:
clinical_data_test = clinical_data[["personReference", "consentDate", "sex", "diagnosisGroups"]].copy().reset_index(drop=True)
print(clinical_data_test.shape)
clinical_data_test.columns = [["personReference", "consentDate"]]
clinical_data_test = clinical_data_test.dropna().reset_index(drop=True)
print(clinical_data_test.shape)
clinical_data_test = clinical_data_test.drop_duplicates().reset_index(drop=True)
print(clinical_data_test.shape)
clinical_data_test["personReference"] = clinical_data_test["personReference"].astype('object')
clinical_data_test["consentDate"] = clinical_data_test["consentDate"].astype('datetime64[ns]')
print(clinical_data_test.shape)
clinical_data_test.head(2)

(1480, 4)
(1468, 4)
(1468, 4)
(1468, 4)


,personReference,consentDate,sex,diagnosisGroups
0,npr446854059583,2010-04-11,MALE,HEMA
1,npr165959402726,2010-06-16,MALE,HEMA


In [165]:
print(exams_data.shape)
exams_data = exams_data.drop_duplicates()
print(exams_data.shape)

(1828, 4)
(1828, 4)


In [166]:
clinical_data_test.dtypes

personReference            object
consentDate        datetime64[ns]
sex                        object
diagnosisGroups            object
dtype: object

In [167]:
clinical_data_test.consentDate

0      2010-04-11
1      2010-06-16
2      2010-07-02
3      2010-09-13
4      2010-09-16
          ...    
1463   2021-09-30
1464   2021-10-06
1465   2021-10-06
1466   2021-10-11
1467   2021-10-11
Name: consentDate, Length: 1468, dtype: datetime64[ns]

In [168]:
clinical_data_test.consentDate.dtype

dtype('<M8[ns]')

In [169]:
exams_data.scan_date.dtype

dtype('<M8[ns]')

In [170]:
clinical_data_test.consentDate.values.dtype

dtype('<M8[ns]')

In [171]:
utils.display_full(clinical_data_test)

      personReference consentDate      sex       diagnosisGroups
0     npr446854059583  2010-04-11     MALE                  HEMA
1     npr165959402726  2010-06-16     MALE                  HEMA
2     npr844745391915  2010-07-02     MALE              LC, HEMA
3     npr351786103610  2010-09-13     MALE                  HEMA
4     npr253202446506  2010-09-16     MALE                  HEMA
5     npr671162790654  2010-09-22     MALE             CRC, HEMA
6     npr685106741724  2010-09-22   FEMALE                  HEMA
7     npr728798676693  2010-10-05     MALE                  HEMA
8     npr824033832482  2010-10-08   FEMALE                  HEMA
9     npr972308330752  2010-10-12   FEMALE                  HEMA
10    npr411742927511  2010-10-12     MALE                  HEMA
11    npr694994026946  2010-10-12     MALE                  HEMA
12    npr548114351603  2010-10-22     MALE                  HEMA
13    npr491098418481  2010-10-25     MALE                  HEMA
14    npr330543079169  20

In [172]:
exams_data.dtypes

patient_directory            object
system                       object
npr                          object
scan_date            datetime64[ns]
dtype: object

In [173]:
exams_data.scan_date.dtype

dtype('<M8[ns]')

In [174]:
linked_data = pd.merge(exams_data, clinical_data_test, how="inner", left_on=["npr", "scan_date"], right_on=["personReference", "consentDate"], suffixes=['','_'])
linked_data

,patient_directory,system,npr,scan_date,personReference,consentDate,sex,diagnosisGroups
0,ASPTCTX0001_npr351786103610-20100913,ASPTCTX0001,npr351786103610,2010-09-13,npr351786103610,2010-09-13,MALE,HEMA
1,ASPTCTX0001_npr796901355424-20110512,ASPTCTX0001,npr796901355424,2011-05-12,npr796901355424,2011-05-12,FEMALE,HEMA
2,ASPTCTX0001_npr692499291026-20141030,ASPTCTX0001,npr692499291026,2014-10-30,npr692499291026,2014-10-30,MALE,"HEMA, PAT"
3,ASPTCTX0001_npr569132387737-20141030,ASPTCTX0001,npr569132387737,2014-10-30,npr569132387737,2014-10-30,MALE,HEMA
4,ASPTCTX0001_npr481526034495-20141106,ASPTCTX0001,npr481526034495,2014-11-06,npr481526034495,2014-11-06,MALE,HEMA
5,ASPTCTX0001_npr585668131301-20141203,ASPTCTX0001,npr585668131301,2014-12-03,npr585668131301,2014-12-03,FEMALE,HEMA
6,ASPTCTX0001_npr413803357102-20141204,ASPTCTX0001,npr413803357102,2014-12-04,npr413803357102,2014-12-04,MALE,HEMA
7,ASPTCTX0001_npr978627791454-20141209,ASPTCTX0001,npr978627791454,2014-12-09,npr978627791454,2014-12-09,MALE,HEMA
8,ASPTCTX0001_npr897698727289-20150209,ASPTCTX0001,npr897698727289,2015-02-09,npr897698727289,2015-02-09,FEMALE,HEMA
9,ASPTCTX0001_npr209149612467-20150706,ASPTCTX0001,npr209149612467,2015-07-06,npr209149612467,2015-07-06,FEMALE,HEMA


In [175]:
linked_data.shape

(18, 8)

In [179]:
print(exams_data.shape)
print(clinical_data_test.shape)

(1828, 4)
(1468, 4)


In [249]:
linked_data = pd.merge_asof(exams_data, clinical_data_test, left_on=["scan_date"], right_on=["consentDate"], left_by="npr", right_by="personReference",  tolerance=pd.Timedelta("365d"), direction="forward")
print(linked_data.shape)
linked_data.head()

(1828, 8)


,patient_directory,system,npr,scan_date,personReference,consentDate,sex,diagnosisGroups
0,ASPTCTX0001_npr601608371367-20100224,ASPTCTX0001,npr601608371367,2010-02-24,NaN,NaT,NaN,NaN
1,ASPTCTX0001_npr642326741392-20100301,ASPTCTX0001,npr642326741392,2010-03-01,NaN,NaT,NaN,NaN
2,ASPTCTX0001_npr967652562138-20100730,ASPTCTX0001,npr967652562138,2010-07-30,npr967652562138,2011-06-22,MALE,HEMA
3,ASPTCTX0001_npr355725597861-20100816,ASPTCTX0001,npr355725597861,2010-08-16,NaN,NaT,NaN,NaN
4,ASPTCTX0001_npr824033832482-20100902,ASPTCTX0001,npr824033832482,2010-09-02,npr824033832482,2010-10-08,FEMALE,HEMA


In [250]:
linked_data1 = linked_data.dropna().reset_index(drop=True).copy()
print(linked_data1.shape)
#(1777, 8)

(196, 8)


In [272]:
linked_data2 = linked_data1[linked_data1.npr == linked_data1.personReference].copy()
linked_data2 = linked_data2.sort_values(by=["npr", "scan_date"]).reset_index(drop=True)
print(linked_data2.shape)
utils.display_full(linked_data2)

(196, 8)
                        patient_directory       system              npr  scan_date  personReference consentDate      sex  diagnosisGroups
0    ASPTCTX0001_lpr385705046400-20140313  ASPTCTX0001  lpr385705046400 2014-03-13  lpr385705046400  2014-03-18  UNKNOWN             HEMA
1    ASPTCTX0001_lpr415675513429-20190201  ASPTCTX0001  lpr415675513429 2019-02-01  lpr415675513429  2019-02-13  UNKNOWN             HEMA
2    ASPTCTX0001_npr100169878450-20130412  ASPTCTX0001  npr100169878450 2013-04-12  npr100169878450  2013-04-17     MALE             HEMA
3    ASPTCTX0001_npr107605794128-20210520  ASPTCTX0001  npr107605794128 2021-05-20  npr107605794128  2021-05-26     MALE             HEMA
4    ASPTCTX0001_npr113096472044-20110318  ASPTCTX0001  npr113096472044 2011-03-18  npr113096472044  2011-03-22     MALE             HEMA
5    ASPTCTX0001_npr116141176598-20191025  ASPTCTX0001  npr116141176598 2019-10-25  npr116141176598  2019-11-05   FEMALE       HEMA, CTRL
6    ASPTCTX0001_npr13723

In [273]:
linked_data2.npr.nunique()

186

In [278]:
multiple_scans = linked_data2.groupby('npr').agg({'scan_date':'count'}).reset_index()
multiple_scans = multiple_scans[multiple_scans.scan_date>1]
multiple_scans

,npr,scan_date
97,npr547741817630,2
103,npr582168347000,2
122,npr692499291026,2
126,npr719609441439,3
173,npr945780558292,4
180,npr967652562138,3


In [282]:
linked_data3 = linked_data2[~linked_data2.npr.isin(multiple_scans.npr.to_list())].copy()
utils.display_full(linked_data3)

                        patient_directory       system              npr  scan_date  personReference consentDate      sex  diagnosisGroups
0    ASPTCTX0001_lpr385705046400-20140313  ASPTCTX0001  lpr385705046400 2014-03-13  lpr385705046400  2014-03-18  UNKNOWN             HEMA
1    ASPTCTX0001_lpr415675513429-20190201  ASPTCTX0001  lpr415675513429 2019-02-01  lpr415675513429  2019-02-13  UNKNOWN             HEMA
2    ASPTCTX0001_npr100169878450-20130412  ASPTCTX0001  npr100169878450 2013-04-12  npr100169878450  2013-04-17     MALE             HEMA
3    ASPTCTX0001_npr107605794128-20210520  ASPTCTX0001  npr107605794128 2021-05-20  npr107605794128  2021-05-26     MALE             HEMA
4    ASPTCTX0001_npr113096472044-20110318  ASPTCTX0001  npr113096472044 2011-03-18  npr113096472044  2011-03-22     MALE             HEMA
5    ASPTCTX0001_npr116141176598-20191025  ASPTCTX0001  npr116141176598 2019-10-25  npr116141176598  2019-11-05   FEMALE       HEMA, CTRL
6    ASPTCTX0001_npr137231127717-2

In [221]:
exams_npr_list = exams_data.npr.str.strip().to_list()
print(len(exams_npr_list))
exams_npr_set = set(exams_npr_list)
print(len(exams_npr_set))

1828
1078


In [251]:
len(exams_npr_set)

1078

In [ ]:
exams_npr_set

In [222]:
clinical_npr_list = clinical_data_test.personReference.str.strip().to_list()
print(len(clinical_npr_list))
clinical_npr_set = set(clinical_npr_list)
print(len(clinical_npr_set))

1468
1468


In [223]:
len(exams_npr_set.intersection(clinical_npr_set))

388

In [254]:
clinical_data_test[clinical_data_test.personReference.isin(exams_npr_list)]

,personReference,consentDate,sex,diagnosisGroups
2,npr844745391915,2010-07-02,MALE,"LC, HEMA"
3,npr351786103610,2010-09-13,MALE,HEMA
8,npr824033832482,2010-10-08,FEMALE,HEMA
26,npr941333664920,2010-11-12,FEMALE,HEMA
29,npr353131040011,2010-11-24,MALE,HEMA
...,...,...,...,...
1452,npr988864921432,2021-08-31,FEMALE,HEMA
1453,npr102928239238,2021-08-31,FEMALE,HEMA
1459,npr988275348495,2021-09-24,FEMALE,"CRC, HEMA"
1462,npr777311700291,2021-09-27,FEMALE,HEMA


In [ ]:
clinical_data_test['personReference'] + '_' + str(clinical_data_test['consentDate'])

In [260]:
clinical_data_test['patient'] = clinical_data_test.apply(lambda x: x.personReference + '_' + x.consentDate)
clinical_data_test.head(2)

AttributeError: 'Series' object has no attribute 'personReference'

In [268]:
temp = clinical_data_test.groupby('personReference').agg({'consentDate':'count'}).reset_index()
temp[temp.consentDate==1]

,personReference,consentDate
0,lpr191214218946,1
1,lpr385705046400,1
2,lpr415675513429,1
3,npr100169878450,1
4,npr101652198453,1
...,...,...
1463,npr998371228388,1
1464,npr998649780071,1
1465,npr999058917244,1
1466,npr999062108241,1
